# Overview

This notebook goes through the pandas API using some practical examples. It is by no means exhaustive.

# Motivation

Searching old code and online documentation for how previous problems were solved is time consuming. Keeping a living document and tagging cells with keywords can help bring about solutions faster.

In [44]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

'en_US.UTF8'

# [Raise on chained assignment](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-view-versus-copy)

In [45]:
# notebooks should begin with this so we do not accidentally mutate the original dataframe
pd.set_option('mode.chained_assignment','raise')

# First, lets read in the data with `read_csv`:

NIH data for 2019 was imported from [exporter.nih.gov](https://exporter.nih.gov/ExPORTER_Catalog.aspx?sid=5&index=0). An [explanation of the fields](https://exporter.nih.gov/about.aspx) is helpful for analysis. We took a random sample of 15k rows so this demo could be lightweight.

- Explicitly pass `header=0` to be able to replace existing column names.
- pass `dtype={ 'ORG_ZIPCODE': 'str'}` to prevent this column from being inferred as an int

In [46]:
column_names=["ID","ACTIVITY","IC","ORG_CITY","ORG_COUNTRY","ORG_DEPT","ORG_NAME","ORG_STATE","ORG_ZIPCODE","PROJECT_START","PROJECT_END","PROJECT_TITLE","DIRECT_COST_AMT","TOTAL_COST"]
df = pd.read_csv('./nih_2019.gzip', header=0, names=column_names, parse_dates=["PROJECT_START","PROJECT_END"], compression='gzip', dtype={ 'ORG_ZIPCODE': 'str'}, encoding="ISO-8859-1")

In [47]:
df.head()

,ID,ACTIVITY,IC,ORG_CITY,ORG_COUNTRY,ORG_DEPT,ORG_NAME,ORG_STATE,ORG_ZIPCODE,PROJECT_START,PROJECT_END,PROJECT_TITLE,DIRECT_COST_AMT,TOTAL_COST
0,9757776,K24,DK,BIRMINGHAM,UNITED STATES,INTERNAL MEDICINE/MEDICINE,UNIVERSITY OF ALABAMA AT BIRMINGHAM,AL,352940001,2018-08-07,2023-07-31,Mentored patient-oriented research in disparit...,149851.0,161839.0
1,9741741,U54,HG,REDEMPTION CITY,NIGERIA,NaN,REDEEMER'S UNIVERSITY,NaN,20001,NaT,NaT,Education and Outreach Core,105350.0,NaN
2,9783793,P30,DK,BRONX,UNITED STATES,NaN,ALBERT EINSTEIN COLLEGE OF MEDICINE,NY,104611900,NaT,NaT,New York Regional Center for Diabetes Translat...,55264.0,NaN
3,9808153,R21,AG,PISCATAWAY,UNITED STATES,NONE,RBHS-ROBERT WOOD JOHNSON MEDICAL SCHOOL,NJ,88548021,2019-08-15,2021-04-30,Evaluation of the lysosomal protease tripeptid...,125000.0,198750.0
4,9816755,R01,AI,MEMPHIS,UNITED STATES,NaN,ST. JUDE CHILDREN'S RESEARCH HOSPITAL,TN,381053678,1997-05-01,2024-05-31,Non-canonical functions of autophagy proteins,250000.0,448750.0


# Lets say we only want to examine US awards.

Always use **`copy()`** whenever you assign a **variable to a slice** of your dataframe **when that slice could be used for anything other than just reading**. This way we will avoid any embarrassing [warnings/exceptions regarding setting view on copy](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy). 

In [48]:
# Always use `copy()` whenever you assign a variable to a slice of your dataframe when that slice could be used for anything other than just reading.
us = df.loc[df['ORG_COUNTRY'] == 'UNITED STATES'].copy()

# Now those zip codes are going to be used later to lookup latitude/longitude. Make sure they contain only digits

In [49]:
# Find rows that contain any non-digit in column
not_okay_zips = us.loc[us['ORG_ZIPCODE'].str.contains('\D', regex=True), ['ID', 'ORG_CITY', 'ORG_NAME', 'ORG_STATE', 'ORG_ZIPCODE', 'TOTAL_COST']]

In [50]:
def highlight_columns(val, match_strings_list):
    v = 'background-color: %s; font-weight: %s;' % ('#ff677d', 'bold')
    t = 'font-weight: normal'
    if (val.name in match_strings_list):
        return [v for i in val.items()]
    else:
        return [t for i in val.items()]

# Sometimes highlighting a column is helpful

In [51]:
# highlight a column by using style.apply and passing a function
not_okay_zips.head().style.apply(highlight_columns, match_strings_list=['ORG_ZIPCODE'])

,ID,ORG_CITY,ORG_NAME,ORG_STATE,ORG_ZIPCODE,TOTAL_COST
4423,10055828,LAUREL,nan,MD,"20,707,359",29214.000000


# Oh no, `20,707,359,` does not look like a valid zip code.

## Since we just have one baddie, lets correct it using `.loc` -- passing a boolean array and a single column label, `ORG_ZIPCODE`.

In [ ]:
us.loc[us['ID'] == 10055828, 'ORG_ZIPCODE'] = '20707' # Would have got SettingWithCopyError if we did not use `copy()` above when assigning a slice of `df` to `us`

# Whew, better. 

## Wait, what about length? Zip codes should only be 5 or 9 characters. It's probably fine, but lets just check really quick:

In [ ]:
us.loc[~(us['ORG_ZIPCODE'].str.len() == 5) & ~(us['ORG_ZIPCODE'].str.len() == 9)].head().style.apply(highlight_columns, match_strings_list=['ORG_ZIPCODE'])

# Yikes, that cannot be right. Fret not, we will smote and vanish these dragons!

## Looks like the data came with leading zeros pre-stripped from the `ORG_ZIPCODE` column. That is going to make joining a little harder so lets go ahead and fix these.

# Pad the zips with `len() < 5` first

In [ ]:
is_len_less_five = us['ORG_ZIPCODE'].str.len() < 5
us.loc[is_len_less_five, 'ORG_ZIPCODE'] = [s.rjust(5, '0')[:5] for idx, s in us.loc[is_len_less_five, 'ORG_ZIPCODE'].items()]

# Pad the zips with `len() > 5` next.

In [ ]:
is_len_gt_five = us['ORG_ZIPCODE'].str.len() > 5
us.loc[(is_len_gt_five), 'ORG_ZIPCODE'] = [s.rjust(9, '0')[:5] for idx, s in us.loc[(is_len_gt_five), 'ORG_ZIPCODE'].items()]

# Lets see if those dragons are gone?!?

In [ ]:
us.loc[~((us['ORG_ZIPCODE'].str.len() == 5) | (us['ORG_ZIPCODE'].str.len() == 9))].size == 0

# Import ZIP to GEO data

The zip/geo data was [downloaded from ODS](https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6InVzLXppcC1jb2RlLWxhdGl0dWRlLWFuZC1sb25naXR1ZGUiLCJvcHRpb25zIjp7fX0sImNoYXJ0cyI6W3siYWxpZ25Nb250aCI6dHJ1ZSwidHlwZSI6ImNvbHVtbiIsImZ1bmMiOiJBVkciLCJ5QXhpcyI6ImxhdGl0dWRlIiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoiI0ZGNTE1QSJ9XSwieEF4aXMiOiJzdGF0ZSIsIm1heHBvaW50cyI6NTAsInNvcnQiOiIifV0sInRpbWVzY2FsZSI6IiIsImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9&location=3,43.19717,-48.51562&basemap=jawg.streets) (Open Data Share)

In [ ]:
zips_to_coords = pd.read_csv('./zip_geo.gzip', sep=';', compression='gzip', dtype={'Zip': 'str'}, encoding="ISO-8859-1")

# Lets make sure there are no dups

In [ ]:
zips_to_coords[zips_to_coords.duplicated('Zip')]

# Wait, there are some coordinates missing. We need to append to the dataframe

In [ ]:
# append an array of series to a dataframe
zip_cols = zips_to_coords.columns
zips_to_add = [pd.Series(['94158', 37.77244949, -122.39166260], index=zip_cols),
               pd.Series(['10065', 40.76429569, -73.96246150], index=zip_cols)]
zips_to_coords = zips_to_coords.append(zips_to_add, ignore_index=True)

In [ ]:
zips_to_coords[zips_to_coords.duplicated('Zip', keep='first')]

# Oh I was wrong, those did not need to be added. Lets remove:

In [ ]:
# drop duplicates in place
zips_to_coords.drop_duplicates(subset="Zip", inplace=True)

In [ ]:
zips_to_coords[zips_to_coords.duplicated('Zip', keep='first')].size == 0

# Whew. That was glorious. Now lets merge these dataframes so we can get lat/long on our `us` dataframe

In [ ]:
# join two dataframes on the original dataframes' column. Index must be set appropriately on second df.
us = us.join(zips_to_coords.loc[:, ['Zip', 'Latitude', 'Longitude']].set_index('Zip'), on='ORG_ZIPCODE')

In [ ]:
us.iloc[:5, [0,14,15]].style.apply(highlight_columns, match_strings_list=['Longitude', 'Latitude'])

## Lets examine top administering agencies

In [ ]:
us['IC'].value_counts()[:5]

## Note that value_counds is a shortcut for `groupby()` and `count()`:

Only `value_counts()` automatically sorts the list though.

In [ ]:
us.groupby('IC').IC.count().sort_values(ascending=False)[:5]

### There are a lot of other cool things you can do with `groupby` when it is combined with the `agg` function.

# Let's say we want to group cities together and get the sum and max direct cost:

In [ ]:
us.groupby('ORG_CITY').DIRECT_COST_AMT.agg([sum, max]).sort_values(by="sum", ascending=False)[:10]

## Lets say we are only interested in 3 NIH agencies.

In [ ]:
nih_agencies = [{
    "code": "AI",
    "name": "NIH National Institute of Allergy and Infectious Diseases (NIAID)"
  }, {
    "code": "GM",
    "name": "NIH National Institute of General Medical Sciences (NIGMS)"
  }, {
    "code": "CA",
    "name": "NIH National Cancer Institute (NCI)"
  }]

In [ ]:
agency_list = [obj['code'] for obj in nih_agencies]

# Create `agc` which contains only grants from AI, GM, CA

In [ ]:
%%time
agc = us.loc[us['IC'].isin(agency_list)].copy()

In [ ]:
agc.loc[:, 'TOTAL_COST'].sum()

## How many grant awards have no associated cost information?

In [ ]:
agc.loc[pd.isna(agc['DIRECT_COST_AMT']) & pd.isna(agc['TOTAL_COST'])].style.apply(highlight_columns, match_strings_list=['DIRECT_COST_AMT', 'TOTAL_COST'])

In [ ]:
# Get a series and count the values grouped by year when column is datetime64[ns]
project_start = agc.loc[:, "PROJECT_START"].groupby(agc.loc[:, "PROJECT_START"].dt.year).count()
ps_series = pd.Series(project_start.values, project_start.index.astype('int64'), name="start")

In [ ]:
project_end = agc.loc[:, "PROJECT_END"].groupby(agc.loc[:, "PROJECT_END"].dt.year).count()
pe_series = pd.Series(project_end.values, project_end.index.astype('int64'), name="end")

In [ ]:
matplotlib.style.use('fivethirtyeight')

In [ ]:
# Create a dataframe from array of series and plot
pd.DataFrame([ps_series, pe_series]).T.plot(kind="bar", width=1.0, figsize=(18, 4), title="Project Start and End Count by Year")

In [ ]:
agc[['TOTAL_COST', 'DIRECT_COST_AMT']].describe()

In [ ]:
largest_awards = agc.loc[:, ["ID", "IC", "ORG_CITY", "ORG_NAME", "PROJECT_TITLE", "TOTAL_COST"]].sort_values(by=['TOTAL_COST'], ascending=False)

In [ ]:
largest_awards.head(10)

# What organizations receive the most awards?

### Create a series with `value_counts()` that sorts the organizations by award count.

In [ ]:
top_organizations = agc['ORG_NAME'].value_counts()

In [ ]:
top_organizations[0:10]